In [1]:
import sys
# sys.path.append('C:\Python27\Lib\site-packages')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import copy
from control import matlab
import time
import cvxopt
cvxopt.solvers.options['show_progress'] = False
import termios
import keyboard 
import serial 
import struct
import numpy as np
from xbee import XBee

In [2]:
class reference():
    
    def __init__(self, horizon_length,x_len, dt = 0.01):
        self.t = 0.
        self.x_ref = np.zeros((x_len,horizon_length))
        self.dt = dt
        self.l = horizon_length

        
    def update_reference(self):
        for i in range(self.l):
            self.x_ref[:, i] = np.array([[0],
                                         [0],
                                         [np.pi/6*math.sin(np.pi/4*(self.t+self.dt*i))],
                                         [0]]).flatten()
        self.t += self.dt
            

In [3]:
def mpc_calculation(A, B, Q, R, x0, horizon, u_len, u_max, x_max, x_des):
    num_states = x0.size*horizon
    num_control = u_len*horizon
    num_vars = num_states+num_control
    
    # Contruction of required martrices
    Q1 = np.zeros([num_vars, num_vars])
    A1 = np.zeros([num_states, num_vars])
    b1 = np.zeros([num_states])
    b1[:x0.size] = -A.dot(x0).flatten()  
    
    q1 = np.zeros([num_vars])
    
    for i in range(horizon):
        Q1[x0.size*i:x0.size*(i+1), x0.size*i:x0.size*(i+1)] = Q
        Q1[num_states+u_len*i:num_states+u_len*(i+1),
           num_states+u_len*i:num_states+u_len*(i+1)] = R
        
        q1[x0.size*i:x0.size*(i+1)] = -x_des[:,i].dot(Q)
        A1[x0.size*i:x0.size*(i+1), num_states+u_len*i:num_states+u_len*(i+1)] = B
        if i>0:
            A1[x0.size*i:x0.size*(i+1), x0.size*(i-1):x0.size*(i+1)] = np.hstack((A, -np.eye(x0.size)))
        else:
            A1[x0.size*i:x0.size*(i+1), x0.size*(i):x0.size*(i+1)] = -np.eye(x0.size)
            
    G = np.zeros([2*num_control, num_vars])
    G[:, num_states:] = np.vstack((np.eye(num_control), -np.eye(num_control)))
    G = np.vstack((G, np.eye(num_states, num_vars)))
    G = np.vstack((G, -np.eye(num_states, num_vars)))
    h = np.vstack((np.tile(u_max, horizon), np.tile(u_max, horizon))).flatten()
    tt = np.vstack((np.tile(x_max, horizon), np.tile(x_max, horizon))).flatten()
    h = np.hstack((h.T, tt.T)).astype(float)
    Q = cvxopt.matrix(Q1)
    p = cvxopt.matrix(q1)
    G = cvxopt.matrix(G)
    h = cvxopt.matrix(h)
    A = cvxopt.matrix(A1)
    b = cvxopt.matrix(b1)
    sol = cvxopt.solvers.qp(Q, p, G, h, A, b)
    x = np.array(sol['x'])
    u = x[num_states:num_states+u_len]
    return u

In [4]:
class system():
    
    def __init__(self, A, B, C, D = None, init_states = None):
        
        self.A = A
        self.B = B
        self.C = C
        
        if D is not None:
            self.D = D
        
        self.x = np.zeros(self.A.shape[0])
        if init_states is not None:
            self.x = copy.deepcopy(init_states)
   
        self.his_x = [init_states]

        
        
    def update_state(self, u, dt = 0.01):
        self.temp_x = self.x.reshape(-1,1)
        self.temp_u = u.reshape(-1,1)
    
        # Calculating the coefficients for Range Kutta Method
        f1 = dt*(self.A.dot(self.temp_x)+self.B.dot(self.temp_u))
        f2 = dt*(self.A.dot(self.temp_x+f1/2)+ self.B.dot(self.temp_u))
        f3 = dt*(self.A.dot(self.temp_x+f2/2)+ self.B.dot(self.temp_u))
        f4 = dt*(self.A.dot(self.temp_x+f3)+ self.B.dot(self.temp_u))
        self.x = self.x + (f1+2*f2+2*f3+f4)/6
        temp_state = np.array(self.x)
        self.his_x.append(temp_state)

In [5]:
def set_transmission():
    port = '/dev/ttyUSB1'
    serial_port = serial.Serial(port,9600)
    serial_port.close()
    serial_port.open()
    time.sleep(0.05)
    return serial_port

In [6]:
def transmission_function(u, serial_port):
    serial_port.write(struct.pack('>B',2))
    serial_port.write(struct.pack('>B',u))
    time.sleep(0.05)

In [7]:
def stop_car(serial_port):
    serial_port.write(struct.pack('>B',1))
    serial_port.write(struct.pack('>B',128+30))
    time.sleep(0.05)

In [8]:
def get_reference(horizon_length, i):
    
#     The reference states for constant reference tracking is
    x_des = np.tile(np.array([[0],
                              [0],
                              [0],
                              [0]]), horizon_length)
    
    x_des [0,:] = [0,2.93, 3.19, 37.65, 1.22, 0, -19.20, 0.47, 0, 0.74];

    if(i<=50):
        obstacle_presence = True
        
    else:
        obstacle_presence = False
        
    i-=1
#     print(i)
    
    return x_des, obstacle_presence, i

In [9]:
def main():
    dt = 0.005
    simulation_time = 1
    iter_num = int(simulation_time/dt)
    j = 120
    m = 1570
    Iz = 2870
    lf = 1.2
    lr = 1.6
    cf = 2*19000
    cr = 2*33000
    vx = 5
    
    A = np.array([[0, 1, 0, 0],
                 [0, -(cr+cf)/(m*vx), 0, -vx + (cr*lr-cf*lf)/(m*vx)],
                 [0, 0, 0, 1],
                 [0, (cf*lf-cr*lr)/Iz/vx,0, -(cr*lr**2+cf*lf**2)/(Iz*vx)]])
    
    B = np.array([[0],
                 [cf/(m)],
                 [0],
                 [cf*lf/Iz]])
    
    C = np.eye(4)
    
    D = np.zeros((4,1))
    
    # simulate the contiuous matrix
    init_x = np.array([[0], [0], [0], [0]])
    plant = system(A, B, C, init_states = init_x)
    
    # create the continuous system
    sysa = matlab.ss(A, B, C, D)
    
    sysd = matlab.c2d(sysa, dt)
    
    Ad = sysd.A
    Bd = sysd.B
    
    # Adding the wieght for the cost function
    Q = np.diag([1000, 1, 1, 1])
    R = np.diag([1])
    horizon_length = 10
    input_limit = np.array([np.pi/6])
    x_max = np.array([2, 1, np.pi/6, np.pi/12])

#     The reference states for constant reference tracking is
#     x_des = np.tile(np.array([[0],
#                               [0],
#                               [0],
#                               [0]]), horizon_length)
#     x_des[3,:] = [0,2.93, 3.19, 37.65, 1.22, 0, -19.20, -15.47, 0, -69.74];
    ref1 = reference(horizon_length, len(A), dt = dt)
    u_len = len(B[0])
    ulist = []
    
    serial_port = set_transmission()
    
    for i in range(iter_num):
        start_time = time.time()
        states = plant.x.reshape(-1,1)
#         ref1.update_reference()
        x_des, obstacle_presence, distance = get_reference(horizon_length, j)
        j-=1
#         print(x_des.shape, obstacle_presence, distance)
        if(not ((obstacle_presence == True) and (distance <=10))):
            opt_u = mpc_calculation(Ad, Bd, Q, R, states, horizon_length, u_len, input_limit, x_max, x_des)
            transmission_function(int(opt_u[0][0]*180/np.pi)+128+30, serial_port)
            
        else:
            opt_u = mpc_calculation(Ad, Bd, Q, R, states, horizon_length, u_len, input_limit, x_max, x_des)
            opt_u[0][0] = 0
            stop_car(serial_port)
        
        ulist.append(int(opt_u[0][0]*180/np.pi))

        plant.update_state(opt_u, dt = dt)


In [10]:
main()

simulation time = 0
simulation time = 1
simulation time = 2
simulation time = 3
simulation time = 4
simulation time = 5
simulation time = 6
simulation time = 7
simulation time = 8
simulation time = 9
simulation time = 10
simulation time = 11
simulation time = 12
simulation time = 13
simulation time = 14
simulation time = 15
simulation time = 16
simulation time = 17
simulation time = 18
simulation time = 19
simulation time = 20
simulation time = 21
simulation time = 22
simulation time = 23
simulation time = 24
simulation time = 25
simulation time = 26
simulation time = 27
simulation time = 28
simulation time = 29
simulation time = 30
simulation time = 31
simulation time = 32
simulation time = 33
simulation time = 34
simulation time = 35
simulation time = 36
simulation time = 37
simulation time = 38
simulation time = 39
simulation time = 40
simulation time = 41
simulation time = 42
simulation time = 43
simulation time = 44
simulation time = 45
simulation time = 46
simulation time = 47
si

KeyboardInterrupt: 